In [ ]:
## Import packages

from collections import OrderedDict

import numpy as np

import torch

from torch.utils.data import DataLoader
from torch.distributions import Normal

from eeyore.data import Iris
from eeyore.models import mlp
from eeyore.mcmc import MetropolisHastings, MALA, PowerPosteriorSampler

from timeit import default_timer as timer
from datetime import timedelta

In [ ]:
## Load iris data

iris = Iris()
dataloader = DataLoader(iris, batch_size=150)

In [ ]:
## Setup MLP model

hparams = mlp.Hyperparameters(dims=[4, 3, 3])
model = mlp.MLP(hparams=hparams)
model.prior = Normal(torch.zeros(model.num_params(), dtype=model.dtype), np.sqrt(3)*torch.ones(model.num_params(), dtype=model.dtype))

In [ ]:
## Setup PowerPosteriorSampler

theta0 = model.prior.sample()
# per_chain_samplers = [['MALA', {'step': 0.025}], ['MALA', {'step': 0.025}], ['MALA', {'step': 0.025}]]
per_chain_samplers = 11 * [['MALA', {'step': 0.025}],]

In [ ]:
sampler = PowerPosteriorSampler(model, theta0, dataloader, per_chain_samplers)

In [ ]:
## Run PowerPosteriorSampler

start_time = timer()

sampler.run(num_iterations=11000, num_burnin=1000)

end_time = timer()
print("Time taken: {}".format(timedelta(seconds=end_time-start_time)))

In [ ]:
sampler.categoricals[4].probs

In [ ]:
torch.sum(sampler.categoricals[4].probs)

In [ ]:
torch.log(sampler.categoricals[4].probs[sampler.from_events_to_seq(10, 4)]).item()

In [ ]:
sampler.categoricals[4].log_prob(torch.tensor(9))

In [ ]:
sampler.categoricals[4].log_prob(torch.tensor(sampler.from_events_to_seq(10, 4)))

In [ ]:
sampler.categoricals[4].sample()

In [ ]:
[sampler.sample_categorical(4) for _ in range(10)]

In [ ]:
x = torch.tensor([2., 3.])

In [ ]:
x.detach_()

In [ ]:
y = x.clone().detach()

In [ ]:
y